In [16]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [17]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [18]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 733958e7
{
    "uid": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "2a6812b0-9b95-4336-8819-26889a677b18",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "30eb91f9-d82c-4d1a-9370-9461ac531b92"
}


In [19]:
task_name = "Jupyter Wind Thresholds"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 810f588b
{
    "uid": "810f588b-f0df-44e2-bf22-008c9d052d04",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:17:01.757579",
    "config": {
        "case_dir": "foam",
        "task_type": "parent"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_b01e2da8",
    "parent": null,
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Wind Thresholds",
    "dependent_on": null
}


In [20]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read())
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [21]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: 4c679c8e
{
    "uid": "4c679c8e-bd51-4c90-868c-d513954c7322",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:17:01.967950",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": {
            "surfaces": {
                "_269c2d39-4173-45a4-8f97-a21f3917d174": {
                    "level": {
                        "max": "2",
                        "min": "2"
       

In [22]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: 61e9d5ef
{
    "uid": "61e9d5ef-624c-424d-a3f3-985d1f57cfdd",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:17:02.278808",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_b01e2da8",
    "parent": {
        "uid": "810f588b-f0df-44e2-bf22-008c9d052d04",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Mesh",
    "dependent_on": {
        "uid": "4c679c8e-bd51-4c90-868c-d513954c7322",
        "name": "Actions"
    }
}


In [23]:
payload_vwt = {
    "name": "VWT",
    "config": {
        "task_type": "cfd",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            45,
            90,
            135,
            180,
            225,
            270,
            315,
        ],
        "cpus": cpus,
        "iterations": {
            "init": 100,
            "run": 50
        }
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

vwt_task = client.create(f"/api/project/{project_id}/task/", payload_vwt)
vwt_id = vwt_task.get("uid")

print(f"VWT TASK: {vwt_id[:8]}")
print(json.dumps(vwt_task, indent=4))

VWT TASK: c3a5ee92
{
    "uid": "c3a5ee92-33dd-4fa1-b150-cd6ace310c08",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:17:02.415301",
    "config": {
        "cmd": "wind_tunnel",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            0,
            45,
            90,
            135,
            180,
            225,
            270,
            315
        ],
        "task_type": "parent",
        "iterations": {
            "run": 50,
            "init": 100
        }
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_b01e2da8",
    "parent": {
        "uid": "810f588b-f0df-44e2-bf22-008c9d052d04",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "VWT",
    "dependent_on": null
}


In [24]:
probe_sets_path = os.path.join(data_path, "mesh_vwt", "probe_sets.json")
with open(probe_sets_path, 'r') as f:
    sets = json.load(f)

payload = {
    "name": "Probing",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [1, 1, 1],
        "sets": sets,
        "fields": ["Utrans", ]
    },
    "parent": parent_id,
    "dependent_on": vwt_id
}
probe_task = client.create(f"/api/project/{project_id}/task/", payload)
probe_id = probe_task.get("uid")

print(f"PROBE TASK: {probe_id[:8]}")
print(json.dumps(probe_task, indent=4))


PROBE TASK: 885f38e9
{
    "uid": "885f38e9-a2fd-4794-a0c4-5b47327008c2",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:17:03.060168",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            1,
            1,
            1
        ],
        "sets": [
            {
                "name": "set1",
                "points": [
                    [
                        -34.02777862548828,
                        -34.02777862548828,
                        1.5
                    ],
                    [
                        -34.02777862548828,
                        -32.083335876464844,
                        1.5
                    ],
                    [
                        -34.02777862548828,
                        -30.13888931274414,
                        1.5
                    ],
                    [
                        -34.02777862548828,
                        -28.194442749023438,
           

In [27]:
epw_name = "DNK_Copenhagen.061800_IWEC.epw"
epw_path = os.path.join(data_path, "mesh_vwt", epw_name)
file_upload = client.update(f"api/task/{parent_id}/file/WeatherFiles/{epw_name}/",
                            files=open(epw_path, 'r').read())
print(f"UPLOADED EPW FILE: {file_upload.get('file')}")

payload = {
    "name": "Wind Thresholds",
    "config": {
        "task_type": "cfd",
        "cmd": "run_wind_thresholds",
        "case_dir": "VWT/",
        "cpus": [2, 1, 1],
        "patches": ["set1"],
        "epw_file": f"WeatherFiles/{epw_name}",
        "set_foam_patch_fields": False
    },
    "parent": parent_id,
    "dependent_on": probe_id
}
threshold_task = client.create(f"/api/project/{project_id}/task/", payload)
threshold_id = threshold_task.get("uid")

print(f"WIND THRESHOLD TASK: {threshold_id[:8]}")
print(json.dumps(threshold_task, indent=4))


UPLOADED EPW FILE: WeatherFiles/DNK_Copenhagen.061800_IWEC.epw
WIND THRESHOLD TASK: 96c2bbe8
{
    "uid": "96c2bbe8-4bf0-4464-b25d-477dc98fe50a",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-28T13:37:05.856086",
    "config": {
        "cmd": "run_wind_thresholds",
        "cpus": [
            2,
            1,
            1
        ],
        "patches": [
            "set1"
        ],
        "case_dir": "VWT/",
        "epw_file": "WeatherFiles/DNK_Copenhagen.061800_IWEC.epw",
        "task_type": "cfd",
        "set_foam_patch_fields": false
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_b01e2da8",
    "parent": {
        "uid": "810f588b-f0df-44e2-bf22-008c9d052d04",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Wind Thresholds",
    "dependent_on": {
        "uid": "885f38e9-a2fd-4794-a0c4-5b473

In [26]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")